In [1]:
import os

os.chdir("/root/dev/DMTet_Models/")
os.getcwd()

'/root/dev/DMTet_Models'

In [2]:
import torch

torch.cuda.set_device(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!gpustat

aa4173e0a5f2                Fri May 10 23:19:43 2024  535.129.03
[0] NVIDIA GeForce RTX 4090 | 25°C,   0 % |  1105 / 24564 MB |
[1] NVIDIA GeForce RTX 4090 | 52°C,  94 % | 13436 / 24564 MB |
[2] NVIDIA GeForce RTX 4090 | 50°C,  99 % | 21440 / 24564 MB |
[3] NVIDIA GeForce RTX 4090 | 47°C,  73 % | 12657 / 24564 MB |
[4] NVIDIA GeForce RTX 4090 | 54°C,  99 % | 20212 / 24564 MB |
[5] NVIDIA GeForce RTX 4090 | 50°C,  99 % | 20270 / 24564 MB |


In [30]:
from easydict import EasyDict

# Explore mesh

In [39]:
def initial_guess_material_knownkskd(geometry, mlp, FLAGS, init_mat=None):
    nrm_min, nrm_max = torch.tensor(FLAGS.nrm_min, dtype=torch.float32, device='cuda'), torch.tensor(FLAGS.nrm_max, dtype=torch.float32, device='cuda')

    if mlp:
        mlp_min = nrm_min
        mlp_max = nrm_max
        mlp_map_opt = mlptexture.MLPTexture3D(geometry.getAABB(), channels=3, min_max=[mlp_min, mlp_max])
        # mlp_map_opt = mlptexture.MLPTexture3D(geometry.getAABB(), channels=3, min_max=None)
        mat =  material.Material({
            'kd'     : init_mat['kd'],
            'ks'     : init_mat['ks'],
            'normal' : mlp_map_opt,
        })
    else:
        # Setup normal map
        if FLAGS.random_textures or init_mat is None or 'normal' not in init_mat:
            normal_map_opt = texture.create_trainable(np.array([0, 0, 1]), FLAGS.texture_res, not FLAGS.custom_mip, [nrm_min, nrm_max])
        else:
            normal_map_opt = texture.create_trainable(init_mat['normal'], FLAGS.texture_res, not FLAGS.custom_mip, [nrm_min, nrm_max])

        mat = material.Material({
            'kd'     : init_mat['kd'],
            'ks'     : init_mat['ks'],
            'normal' : normal_map_opt
        })

    if init_mat is not None:
        mat['bsdf'] = init_mat['bsdf']
    else:
        mat['bsdf'] = 'pbr'

    return mat

In [27]:
# MeshDiffusion
from MeshDiffusion.nvdiffrec.lib.render import material, util, mesh, texture, mlptexture, light, render
from MeshDiffusion.nvdiffrec.lib.geometry.dmtet import DMTetGeometry

In [6]:
mesh_fname = "/root/dev/DMTet_Models/MeshDiffusion/data/02691156/1a04e3eab45ca15dd86060f189eb133/model.obj"

In [46]:
mtl_override = None
normal_only = True # False?

geometry_kwargs = EasyDict({
    "iter": 5000,
    "sdf_regularizer": 0.2,
})

first_stage_deform = 2.0
second_stage_deform = 3.0

mat_kwargs = EasyDict({
    "nrm_min": [-1.0, -1.0, 0.0],
    "nrm_max": [1.0, 1.0, 1.0],
    "random_textures": True,
    "texture_res": [2048, 2048],
    "custom_mip": False
})

dataset_kwargs = EasyDict({
    "train_res": [1000, 1000],
    "random_lgt": True,
    "dataset_flat_shading": False,
    "texture_res": [2048, 2048],
    "envmap": "./MeshDiffusion/nvdiffrec/data_/irrmaps/aerodynamics_workshop_2k.hdr",
    "env_scale": 1.0,
    "cam_near_far": [0.1, 1000.0],
    "iter": 5000,
    "batch": 1,
    "layers": 1
})

In [8]:
mtl_default = {
    "name": "_default_mat",
    "bsdf": "diffuse",
    "uniform": True,
    "kd": texture.Texture2D(torch.tensor([0.75, 0.3, 0.6], dtype=torch.float32, device=device), trainable=False),
    "ks": texture.Texture2D(torch.tensor([0.0, 0.0, 0.0], dtype=torch.float32, device=device), trainable=False)
}

In [10]:
ref_mesh = mesh.load_mesh(mesh_fname, mtl_override, mtl_default, use_default=normal_only, no_additional=True)

Load use-defined default mtl
Skip loading non-default materials
[{'name': '_default_mat', 'bsdf': 'diffuse', 'uniform': True, 'kd': Texture2D(), 'ks': Texture2D()}]


In [22]:
# mesh.Mesh
print("ref_mesh.v_pos.shape: ", ref_mesh.v_pos.shape)
print("ref_mesh.v_nrm.shape: ", ref_mesh.v_nrm.shape)
# print("ref_mesh.v_tex.shape: ", ref_mesh.v_tex.shape) -> None
# print("ref_mesh.v_tng.shape: ", ref_mesh.v_tng.shape) -> None
print("ref_mesh.t_pos_idx.shape: ", ref_mesh.t_pos_idx.shape)
print("ref_mesh.t_nrm_idx.shape: ", ref_mesh.t_nrm_idx.shape)
# print("ref_mesh.t_tex_idx.shape: ", ref_mesh.t_tex_idx.shape) -> None
# print("ref_mesh.t_tng_idx.shape: ", ref_mesh.t_tng_idx.shape) -> None
print("ref_mesh.material.keys(): ", ref_mesh.material.keys())
print(ref_mesh.material["name"], ref_mesh.material["bsdf"], ref_mesh.material["uniform"], ref_mesh.material["kd"], ref_mesh.material["ks"])

ref_mesh.v_pos.shape:  torch.Size([1248, 3])
ref_mesh.v_nrm.shape:  torch.Size([1248, 3])
ref_mesh.t_pos_idx.shape:  torch.Size([4668, 3])
ref_mesh.t_nrm_idx.shape:  torch.Size([4668, 3])
ref_mesh.material.keys():  dict_keys(['name', 'bsdf', 'uniform', 'kd', 'ks'])
_default_mat diffuse True Texture2D() Texture2D()


In [45]:
import numpy as np
import nvdiffrast.torch as dr

ref_mesh = mesh.center_by_reference(ref_mesh, mesh.aabb_clean(ref_mesh), 1.0)

a = ref_mesh.v_nrm.clone()
ref_mesh = mesh.auto_normals(ref_mesh) ###

# dataset_mesh.DatasetMesh
glctx = dr.RasterizeCudaContext
cam_radius = 2.0
fovy = np.deg2rad(45)
aspect = dataset_kwargs.train_res[1] / datset_kwargs.train_res[0]
random_lgt = dataset_kwargs.random_lgt
camera_lgt = False
flat_shading = dataset_kwargs.dataset_flat_shading



# lgt = light.load_env("./MeshDiffusion/nvdiffrec/data_/irrmaps/aerodynamics_workshop_2k.hdr", scale=1.0, trainable=False)

geometry = DMTetGeometry(64, 1.1, geometry_kwargs, root="./MeshDiffusion/nvdiffrec", deform_scale=first_stage_deform)

mat = initial_guess_material_knownkskd(geometry, True, mat_kwargs, mtl_default)

normalization: tensor([0., 0., 0.], device='cuda:0') 1.0
Encoder output: 32 dims


# Colored Pointcloud sampling

In [205]:
!ls /root/dev/DMTet_Models/MeshDiffusion/data/02691156/1a04e3eab45ca15dd86060f189eb133

model.mtl  model.obj


In [200]:
!ls /root/dev/DMTet_Models/MeshDiffusion/data/03001627/2cf960825840947aa903b9ef0ae7fb14/

images	model.mtl  model.obj


In [204]:
!find /root/dev/DMTet_Models/MeshDiffusion/data/03001627 -type d -name "*images" | wc -l
!find /root/dev/DMTet_Models/MeshDiffusion/data/03001627 -type f -name "*.obj" | wc -l

3817
6778


In [208]:
import torch
from pytorch3d.datasets import ShapeNetCore
from pytorch3d.io import load_objs_as_meshes
from pytorch3d.structures import Meshes
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.renderer import TexturesAtlas

mesh_fname = "/root/dev/DMTet_Models/MeshDiffusion/data/03001627/2cf960825840947aa903b9ef0ae7fb14/model.obj"
verts, faces, aux = load_obj(
    mesh_fname,
    load_textures=True,
    create_texture_atlas=True
) # aux: normals, uv coords, mat colors, textures, ...
mesh = Meshes(verts=[verts], faces=[faces.verts_idx], textures=TexturesAtlas(aux.texture_atlas[None]))

In [149]:
print("aux.texture_atlas.shape: ", aux.texture_atlas.shape)
print("aux.texture_images.keys(): ", aux.texture_images.keys())
# print("aux.verts_uvs.shape: ", aux.verts_uvs.shape) → None
print("aux.material_colors.keys(): ", aux.material_colors.keys())

aux.texture_atlas.shape:  torch.Size([4668, 4, 4, 3])
aux.texture_images.keys():  dict_keys([])
aux.material_colors.keys():  dict_keys(['material_1_24', 'material_10_24', 'material_0_24', 'material_2_24', 'material_4_24', 'material_8_24', 'material_9_24', 'material_6_24', 'material_7_24', 'material_5_24', 'material_3_24'])


In [142]:
met_atts = [attr for attr in dir(aux)]
met_atts = [met_att for met_att in met_atts if not met_att.startswith("__") and not met_att.startswith("_")]
print(met_atts)

['count', 'index', 'material_colors', 'normals', 'texture_atlas', 'texture_images', 'verts_uvs']


In [143]:
met_atts = [attr for attr in dir(meshes)]
met_atts = [met_att for met_att in met_atts if not met_att.startswith("__") and not met_att.startswith("_")]
print(met_atts)

['clone', 'cpu', 'cuda', 'detach', 'device', 'edges_packed', 'edges_packed_to_mesh_idx', 'equisized', 'extend', 'faces_areas_packed', 'faces_list', 'faces_normals_list', 'faces_normals_packed', 'faces_normals_padded', 'faces_packed', 'faces_packed_to_edges_packed', 'faces_packed_to_mesh_idx', 'faces_padded', 'get_bounding_boxes', 'get_mesh_verts_faces', 'has_verts_normals', 'isempty', 'laplacian_packed', 'mesh_to_edges_packed_first_idx', 'mesh_to_faces_packed_first_idx', 'mesh_to_verts_packed_first_idx', 'num_edges_per_mesh', 'num_faces_per_mesh', 'num_verts_per_mesh', 'offset_verts', 'offset_verts_', 'sample_textures', 'scale_verts', 'scale_verts_', 'split', 'submeshes', 'textures', 'to', 'update_padded', 'valid', 'verts_list', 'verts_normals_list', 'verts_normals_packed', 'verts_normals_padded', 'verts_packed', 'verts_packed_to_mesh_idx', 'verts_padded', 'verts_padded_to_packed_idx']


In [94]:
!ls /root/dev/DMTet_Models/MeshDiffusion/data

02691156  02958343  03001627  04090263	04379243


In [107]:
shapenet_data = ShapeNetCore(
    "/root/dev/DMTet_Models/MeshDiffusion/data",
    synsets=["03001627"], # ["02691156", "02958343", "03001627", "04090263", "04379243"],
    version=1,
    load_textures=True,
    texture_resolution=8
)
len(shapenet_data)

6778

In [96]:
shapenet_data[0].keys()

dict_keys(['synset_id', 'model_id', 'verts', 'faces', 'textures', 'label'])

In [97]:
torch.ones_like(verts)[None].shape

torch.Size([1, 1248, 3])

In [195]:
from pytorch3d.renderer import TexturesVertex, TexturesAtlas

idx_start = 40
idx_end = 50
verts, faces, textures = [], [], []
for i in range(idx_start, idx_end):
    data = shapenet_data[i]
    verts.append(data["verts"] + i)
    faces.append(data["faces"])
    texture = data["textures"]
    textures.append(texture)
    # texture = TexturesAtlas(texture) # (B, F, R, R, C) → (B, H, W, K, C)
textures = TexturesAtlas(textures)
meshes = Meshes(verts=verts, faces=faces, textures=textures)

/opt/conda/envs/triplora/lib/python3.9/site-packages/pytorch3d/io/mtl_io.py:480: UserWarning:

Texture file does not exist: /root/dev/DMTet_Models/MeshDiffusion/data/03001627/11b7c86fc42306ec7e7e25239e7b8f85/./models/untitled/texture_0.jpg

/opt/conda/envs/triplora/lib/python3.9/site-packages/pytorch3d/io/mtl_io.py:480: UserWarning:

Texture file does not exist: /root/dev/DMTet_Models/MeshDiffusion/data/03001627/11b7c86fc42306ec7e7e25239e7b8f85/./models/untitled/texture.jpg



In [203]:
# https://github.com/facebookresearch/pytorch3d/blob/main/pytorch3d/vis/plotly_vis.py
from pytorch3d.vis import plotly_vis

fig = plot_scene({
    "subplot_title": {
        "mesh_trace_title": mesh
    }
})
fig.show()

In [7]:
from pytorch3d.io import load_objs_as_meshes
import matplotlib.pyplot as plt

wo_img = "/root/dev/DMTet_Models/MeshDiffusion/data/02691156/1a04e3eab45ca15dd86060f189eb133/model.obj"
w_img = "/root/dev/DMTet_Models/MeshDiffusion/data/03001627/2cf960825840947aa903b9ef0ae7fb14/model.obj"
mesh_wo_img = load_objs_as_meshes([w_img])
mesh_w_img = load_objs_as_meshes([wo_img])

In [232]:
mesh_w_img.textures.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [230]:
mesh_wo_img.textures

## with utils

In [14]:
from MeshDiffusion.nvdiffrec.lib.render.obj import load_obj
from MeshDiffusion.nvdiffrec.lib.render import texture, material, mesh


def _find_mat(materials, name):
    for mat in materials:
        if mat['name'] == name:
            return mat
    return materials[0] # Materials 0 is the default


def load_obj(filename, clear_ks=True, mtl_override=None, mtl_default=None, use_default=False, no_additional=False):
    obj_path = os.path.dirname(filename)

    # Read entire file
    with open(filename, 'r') as f:
        lines = f.readlines()

    # Load materials
    if mtl_default is None:
        all_materials = [
            {
                'name' : '_default_mat',
                'bsdf' : 'pbr',
                'kd'   : texture.Texture2D(torch.tensor([0.5, 0.5, 0.5], dtype=torch.float32, device='cuda')),
                'ks'   : texture.Texture2D(torch.tensor([0.0, 0.0, 0.0], dtype=torch.float32, device='cuda'))
            }
        ]
    else:
        print("Load use-defined default mtl")
        all_materials = [mtl_default]

    if not no_additional:
        if mtl_override is None: 
            for line in lines:
                if len(line.split()) == 0:
                    continue
                if line.split()[0] == 'mtllib':
                    all_materials += material.load_mtl(os.path.join(obj_path, line.split()[1]), clear_ks, avoid_pure_black=True) # Read in entire material library
        else:
            all_materials += material.load_mtl(mtl_override)
    else:
        print("Skip loading non-default materials")
    

    # load vertices
    vertices, texcoords, normals  = [], [], []
    for line in lines:
        if len(line.split()) == 0:
            continue
        
        prefix = line.split()[0].lower()
        if prefix == 'v':
            vertices.append([float(v) for v in line.split()[1:]])
        elif prefix == 'vt':
            val = [float(v) for v in line.split()[1:]]
            texcoords.append([val[0], 1.0 - val[1]])
        elif prefix == 'vn':
            normals.append([float(v) for v in line.split()[1:]])
    
    print(all_materials)

    # load faces
    activeMatIdx = None
    used_materials = []
    faces, tfaces, nfaces, mfaces = [], [], [], []
    for line in lines:
        if len(line.split()) == 0:
            continue

        prefix = line.split()[0].lower()
        if prefix == 'usemtl': # Track used materials
            mat = _find_mat(all_materials, line.split()[1])
            if not mat in used_materials:
                used_materials.append(mat)
            activeMatIdx = used_materials.index(mat)
        elif prefix == 'f': # Parse face
            vs = line.split()[1:]
            nv = len(vs)
            vv = vs[0].split('/')
            v0 = int(vv[0]) - 1
            # t1 = int(vv[1]) - 1 if vv[1] != "" else -1
            # n1 = int(vv[2]) - 1 if vv[2] != "" else -1
            try:
                t0 = int(vv[1]) - 1 if vv[1] != "" else -1
                n0 = int(vv[2]) - 1 if vv[2] != "" else -1
            except:
                t0 = n0 = -1
            for i in range(nv - 2): # Triangulate polygons
                vv = vs[i + 1].split('/')
                v1 = int(vv[0]) - 1
                # t1 = int(vv[1]) - 1 if vv[1] != "" else -1
                # n1 = int(vv[2]) - 1 if vv[2] != "" else -1
                try:
                    t1 = int(vv[1]) - 1 if vv[1] != "" else -1
                    n1 = int(vv[2]) - 1 if vv[2] != "" else -1
                except:
                    t1 = n1 = -1
                vv = vs[i + 2].split('/')
                v2 = int(vv[0]) - 1
                # t2 = int(vv[1]) - 1 if vv[1] != "" else -1
                # n2 = int(vv[2]) - 1 if vv[2] != "" else -1
                try:
                    t2 = int(vv[1]) - 1 if vv[1] != "" else -1
                    n2 = int(vv[2]) - 1 if vv[2] != "" else -1
                except:
                    t2 = n2 = -1
                mfaces.append(activeMatIdx)
                faces.append([v0, v1, v2])
                tfaces.append([t0, t1, t2])
                nfaces.append([n0, n1, n2])
    assert len(tfaces) == len(faces) and len(nfaces) == len (faces)

    # Create an "uber" material by combining all textures into a larger texture
    # if len(used_materials) > 1:
    if True:
        uber_material, texcoords, tfaces = material.merge_materials(used_materials, texcoords, tfaces, mfaces)
    elif len(used_materials) == 1:
        uber_material = used_materials[0]
    else:
        uber_material = None

    vertices = torch.tensor(vertices, dtype=torch.float32, device='cuda')
    texcoords = torch.tensor(texcoords, dtype=torch.float32, device='cuda') if len(texcoords) > 0 else None
    normals = torch.tensor(normals, dtype=torch.float32, device='cuda') if len(normals) > 0 else None
    # normals = None
    
    faces = torch.tensor(faces, dtype=torch.int64, device='cuda')
    tfaces = torch.tensor(tfaces, dtype=torch.int64, device='cuda') if texcoords is not None else None
    nfaces = torch.tensor(nfaces, dtype=torch.int64, device='cuda') if normals is not None else None

    print(uber_material)

    # uber_material = all_materials[0]
    # texcoords = normals = tfaces = nfaces = None
    return mesh.Mesh(vertices, faces, normals, nfaces, texcoords, tfaces, material=uber_material)

    # imesh = mesh.Mesh(vertices, faces, normals, nfaces, texcoords, tfaces, material=uber_material)
    # imesh = mesh.auto_normals(imesh)
    # return imesh

In [15]:
mesh_w_img = load_obj(w_img)

[{'name': '_default_mat', 'bsdf': 'pbr', 'kd': Texture2D(), 'ks': Texture2D()}, Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
)]
Material(
  (kd): Texture2D()
  (ks): Texture2D()
)


In [16]:
mesh_wo_img = load_obj(wo_img)

[{'name': '_default_mat', 'bsdf': 'pbr', 'kd': Texture2D(), 'ks': Texture2D()}, Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
), Material(
  (kd): Texture2D()
  (ks): Texture2D()
)]
Material(
  (kd): Texture2D()
  (ks): Texture2D()
)


In [17]:
met_atts = [attr for attr in dir(mesh_wo_img)]
met_atts = [met_att for met_att in met_atts if not met_att.startswith("__") and not met_att.startswith("_")]
print(met_atts)

['clone', 'copy_none', 'f_nrm', 'material', 't_nrm_idx', 't_pos_idx', 't_tex_idx', 't_tng_idx', 'v_nrm', 'v_pos', 'v_tex', 'v_tng']


In [21]:
print("mesh_wo_img.v_pos.shape: ", mesh_wo_img.v_pos.shape)
print("mesh_wo_img.t_pos_idx.shape: ", mesh_wo_img.t_pos_idx.shape)
print("mesh_wo_img.v_tex.shape: ", mesh_wo_img.v_tex.shape)
print("mesh_wo_img.t_tex_idx.shape: ", mesh_wo_img.t_tex_idx.shape)

mesh_wo_img.v_pos.shape:  torch.Size([1248, 3])
mesh_wo_img.t_pos_idx.shape:  torch.Size([4668, 3])
mesh_wo_img.v_tex.shape:  torch.Size([11, 2])
mesh_wo_img.t_tex_idx.shape:  torch.Size([4668, 3])


In [22]:
mesh_wo_img.v_tex[0]

tensor([0., 0.], device='cuda:0')

In [23]:
mesh_wo_img.t_tex_idx[0]

tensor([0, 0, 0], device='cuda:0')

In [24]:
mesh_wo_img.material

Material(
  (kd): Texture2D()
  (ks): Texture2D()
)